In [1]:
import sys
sys.path.append('../')

from gears import PertData, GEARS

/deltadisk/miniconda3/envs/gears/lib/python3.10/site-packages/anndata/utils.py:434: FutureWarning: Importing read_csv from `anndata` is deprecated. Import anndata.io.read_csv instead.
  warnings.warn(msg, FutureWarning)
/deltadisk/miniconda3/envs/gears/lib/python3.10/site-packages/anndata/utils.py:434: FutureWarning: Importing read_excel from `anndata` is deprecated. Import anndata.io.read_excel instead.
  warnings.warn(msg, FutureWarning)
/deltadisk/miniconda3/envs/gears/lib/python3.10/site-packages/anndata/utils.py:434: FutureWarning: Importing read_hdf from `anndata` is deprecated. Import anndata.io.read_hdf instead.
  warnings.warn(msg, FutureWarning)
/deltadisk/miniconda3/envs/gears/lib/python3.10/site-packages/anndata/utils.py:434: FutureWarning: Importing read_loom from `anndata` is deprecated. Import anndata.io.read_loom instead.
  warnings.warn(msg, FutureWarning)
/deltadisk/miniconda3/envs/gears/lib/python3.10/site-packages/anndata/utils.py:434: FutureWarning: Importing read_

Load data. We use norman as an example.

In [2]:
pert_data = PertData('./data')
pert_data.load(data_name = 'adamson')
pert_data.prepare_split(split = 'simulation', seed = 1)
pert_data.get_dataloader(batch_size = 32, test_batch_size = 128)

Found local copy...
Found local copy...
Found local copy...
These perturbations are not in the GO graph and their perturbation can thus not be predicted
['SRPR+ctrl' 'SLMO2+ctrl' 'TIMM23+ctrl' 'AMIGO3+ctrl' 'KCTD16+ctrl']
Local copy of pyg dataset is detected. Loading...
Done!
Local copy of split is detected. Loading...
Simulation split test composition:
combo_seen0:0
combo_seen1:0
combo_seen2:0
unseen_single:21
Done!
Creating dataloaders....
Done!


here1


Create a model object; if you use [wandb](https://wandb.ai), you can easily track model training and evaluation by setting `weight_bias_track` to true, and specify the `proj_name` and `exp_name` that you like.

In [3]:
gears_model = GEARS(pert_data, device = 'cuda:0', 
                        weight_bias_track = False, 
                        proj_name = 'pertnet', 
                        exp_name = 'pertnet')
gears_model.model_initialize(hidden_size = 64, dp=True)                         

Found local copy...


Using DP model!


Load pretrained models: # If use INFL, use_inr=True; else, use_inr=False

'FL_logs' :          use_inr=False, use_lora=False, use_dp=False

'FL_0.5_logs' :      use_inr=False, use_lora=False, use_dp=True

'FL_LoRA_0.5_logs' : use_inr=False, use_lora=True,  use_dp=True

'PPML_logs' :        use_inr=False, use_lora=True,  use_dp=True

'INFL_logs':         use_inr=True,  use_lora=False, use_dp=False


In [4]:
gears_model.load_pretrained('/deltadisk/zhangrongyu/GEARS/demo/results/Ada_INFL_8_logs_0.4', use_inr=False, use_lora=False, use_dp=False)

Using traditional model!
GEARS_Model(
  (pert_w): Linear(in_features=1, out_features=64, bias=True)
  (gene_emb): Embedding(5060, 64, max_norm=True)
  (pert_emb): Embedding(9853, 64, max_norm=True)
  (emb_trans): ReLU()
  (pert_base_trans): ReLU()
  (transform): ReLU()
  (emb_trans_v2): MLP(
    (network): Sequential(
      (0): Linear(in_features=64, out_features=64, bias=True)
      (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Linear(in_features=64, out_features=64, bias=True)
      (4): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (relu): ReLU()
  )
  (pert_fuse): MLP(
    (network): Sequential(
      (0): Linear(in_features=64, out_features=64, bias=True)
      (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Linear(in_features=64, out_features=64, bias=True)
      (4): BatchNorm1d(64, eps=1e-05, momentum=

Make prediction for new perturbation:

In [5]:
gears_model.testing()
gears_model.predict([['FEV'], ['FEV', 'AHR']])

Start Testing...
Best performing model: Test Top 20 DE MSE: 5.3210
Start doing subgroup analysis for simulation split...
test_combo_seen0_mse: nan
test_combo_seen0_pearson: nan
test_combo_seen0_mse_de: nan
test_combo_seen0_pearson_de: nan
test_combo_seen1_mse: nan
test_combo_seen1_pearson: nan
test_combo_seen1_mse_de: nan
test_combo_seen1_pearson_de: nan
test_combo_seen2_mse: nan
test_combo_seen2_pearson: nan
test_combo_seen2_mse_de: nan
test_combo_seen2_pearson_de: nan
test_unseen_single_mse: 0.34889397
test_unseen_single_pearson: 0.6383154809731224
test_unseen_single_mse_de: 5.3210015
test_unseen_single_pearson_de: 0.6507800087075511
test_combo_seen0_pearson_delta: nan
test_combo_seen0_frac_opposite_direction_top20_non_dropout: nan
test_combo_seen0_frac_sigma_below_1_non_dropout: nan
test_combo_seen0_mse_top20_de_non_dropout: nan
test_combo_seen1_pearson_delta: nan
test_combo_seen1_frac_opposite_direction_top20_non_dropout: nan
test_combo_seen1_frac_sigma_below_1_non_dropout: nan
tes

{'FEV': array([-5.6202125e-02, -3.6431472e-10, -1.0457632e-01, ...,
        -1.4842513e-02,  2.2176588e-01, -1.0448498e-01], dtype=float32),
 'FEV_AHR': array([-5.6631666e-02, -3.6431472e-10, -1.1157367e-01, ...,
        -1.4842513e-02,  1.7199016e-01, -1.4565103e-01], dtype=float32)}

Gene list can be found here:

In [6]:
gears_model.gene_list[:5]

['AP006222.2', 'RP11-54O7.16', 'RP11-54O7.1', 'RP11-54O7.3', 'SAMD11']